In [161]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from models import *   
import load_data
import nan_imputation
import helpers

In [162]:
# Reload isdead.py
import importlib
importlib.reload(load_data)
importlib.reload(nan_imputation)
importlib.reload(helpers)

<module 'helpers' from '/Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/helpers.py'>

## Step 0 : Load the Data

In [ ]:
repo_root = helpers.find_repo_root()
repo_root

# load data
# Define the path to the Lifespan folder

#for now test with only control
data_path = os.path.join(repo_root, 'Data/Lifespan')

In [169]:
worms = load_data.load_lifespan(data_path)
#worms.pop("worm_1_companyDrug", None)  # The second argument avoids KeyError if the key doesn't exist

Loading file: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/companyDrug/speeds_and_coordinates_20241016_24_updated.csv
File loaded: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/companyDrug/speeds_and_coordinates_20241016_24_updated.csv
Loading file: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/companyDrug/speeds_and_coordinates_20241016_3_updated.csv
File loaded: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/companyDrug/speeds_and_coordinates_20241016_3_updated.csv
Loading file: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/companyDrug/speeds_and_coordinates_20241016_6_updated.csv
File loaded: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/companyDrug/speeds_and_coordinates_20241016_6_updated.csv
Loading file: /Users/louistschanz/Documents/EPFL-Cours/MA1/

In [165]:
helpers.print_fdict_summary(worms)  

# need to calibrate : 
#companyDrug :
# 1 : 53995
# 2 : 75594
# 3 : 75594
# 4 : 75594
# 5 : 75594
# 6 : 64795
# 21 : 75594
# 22 : 75594
# 23 : 75594
# 24 : 75594
# 25 : 75594
# 26 : 75594
# 
#  control :
# 11 : 75594
# 12 : 75594
# 13 : 75594
# 14 : 75594 
# 15 : 64795
# 16 : 75594
# 31 : 53996
# 32 : 75594
# 33 : 53996
# 34 : 53996
# 35 : 75594
# 36 : 75594


Summary of Loaded Worm Data:
Worm: worm_1
  Shape: (6, 75593)
----------------------------------------
Worm: worm_2
  Shape: (6, 75593)
----------------------------------------
Worm: worm_3
  Shape: (6, 64794)
----------------------------------------
Worm: worm_4
  Shape: (6, 75593)
----------------------------------------
Worm: worm_5
  Shape: (6, 75593)
----------------------------------------
Worm: worm_6
  Shape: (6, 75593)
----------------------------------------
Worm: worm_7
  Shape: (6, 75593)
----------------------------------------
Worm: worm_8
  Shape: (6, 75593)
----------------------------------------
Worm: worm_9
  Shape: (6, 75593)
----------------------------------------
Worm: worm_10
  Shape: (6, 53995)
----------------------------------------
Worm: worm_11
  Shape: (6, 75593)
----------------------------------------
Worm: worm_12
  Shape: (6, 75593)
----------------------------------------
Worm: worm_13
  Shape: (6, 75593)
----------------------------------------
Worm

In [166]:
# Extract data for worm_5
worm_data = worms['worm_5']

# Convert to a Pandas DataFrame
df_worm = pd.DataFrame(
    worm_data.T,  # Transpose the array to align columns properly
    columns=['Frame', 'Speed', 'X', 'Y', 'Changed Pixels', 'Category']  # Add column names
)

# Display the first few rows
print(df_worm.head())

   Frame  Speed   X   Y  Changed Pixels  Category
0    1.0    0.0 NaN NaN             0.0       0.0
1    2.0    0.0 NaN NaN             0.0       0.0
2    3.0    0.0 NaN NaN             0.0       0.0
3    4.0    0.0 NaN NaN             0.0       0.0
4    5.0    0.0 NaN NaN             0.0       0.0


## Step 1 : NaN imputation
> impute only on X and Y columns since only where there are NaN

In [ ]:
for name, lifespan_array in worms.items(): 
    print(f"Processing {name}")
    lifespan_arrayxy = lifespan_array[2:4,:]  # Extract columns for X and Y
    missing_sequences = nan_imputation.count_successive_missing(lifespan_arrayxy)
    for start, end, length in missing_sequences:
        print(f"  Missing sequence starts at column {start}, ends at column {end - 1}, length: {length}")

In [ ]:
print(lifespan_arrayxy)
print(f"Missing sequences for {name}: {missing_sequences}")

In [ ]:
# Rows to check for missing values (2:4 in zero-based indexing)
rows_to_check = slice(2, 4)  # Rows 2 and 3 not row 4

# Apply cut_array to each worm in the dataset
cut_nan_dict = {name: nan_imputation.cut_array(array, rows_to_check) for name, array in worms.items()}

In [ ]:
# Print the shape of the filtered arrays
for name, item in cut_nan_dict.items():
    print(f'{name} : {item.shape}')

In [ ]:
# Extract data for worm_5
worm_data = worms['worm_5']

# Convert to a Pandas DataFrame
df_worm = pd.DataFrame(
    worm_data.T,  # Transpose the array to align columns properly
    columns=['Frame', 'Speed', 'X', 'Y', 'Changed Pixels', 'Category']  # Add column names
)

# Display the first few rows
print(df_worm.head())

# Step 2 : Splitting the Data
> Split the worms in train worms and test worms

In [ ]:
train_worms, test_worms = load_data.split_worms(worms, test_size=0.2)

print(f"Training Worms: {list(train_worms.keys())}")
print(f"Testing Worms: {list(test_worms.keys())}")

# Step 3 : Figure out when do the worms die
>When we find out on which frame he dies, drop the frames after his death

In [ ]:
import isdead
importlib.reload(isdead)

In [ ]:
movement_threshold = 1.0 # Threshold for inactivity detection
processed_worms = {} # Dictionary to store processed worms

dying_times = []

# Iterate through each worm in the dataset
for worm_name, worm_data in worms.items():
    print(f"Processing {worm_name}...")
    # Transpose worm_data for DataFrame creation
    df_worm = pd.DataFrame(worm_data.T,columns=['Frame', 'Speed', 'X', 'Y', 'Changed Pixels', 'Category']) # Transpose the array

    result = isdead.estimate_dying_time(df_worm, movement_threshold) # Use the estimate_dying_time function to find the dying frame
    if result[0] is None:
        print(f"  {worm_name}: No inactivity detected. Retaining all data.")
        processed_worms[worm_name] = worm_data
        continue

    dying_frame, absolute_frame, dying_time_hours, segment_number = result
  
    dying_times.append(dying_time_hours) # Append dying time in hours to the list

    print(f"  {worm_name}: Dying frame = {dying_frame}, Absolute frame = {absolute_frame}, Dying time = {dying_time_hours:.2f} hours, Segment = {segment_number}") # Print details

    # Truncate the data up to the dying frame
    truncated_data = worm_data[:, worm_data[0, :] <= dying_frame]
    processed_worms[worm_name] = truncated_data

# Print summary of processed worms
print("\nSummary of processed worms:")
for name, data in processed_worms.items():
    print(f"{name}: Original frames = {worms[name].shape[1]}, After truncation = {data.shape[1]}")

In [ ]:
# Plot the survival curve
dying_times_sorted = sorted(dying_times) # Sort the dying times in ascending order

# Compute the survival rate
survival_rate = [1 - (i / len(dying_times_sorted)) for i in range(len(dying_times_sorted))]

# Plot the survival curve
plt.figure(figsize=(8, 5))
plt.plot(dying_times_sorted, survival_rate, marker='o', linestyle='-', color='blue')
plt.xlabel('Dying Time (Hours)')
plt.ylabel('Survival Rate')
plt.title('Survival Curve')
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# Visualization

# Prepare data for the plot
original_lengths = [worms[name].shape[1] for name in worms.keys()]
truncated_lengths = [processed_worms[name].shape[1] for name in processed_worms.keys()]
worm_ids = list(worms.keys())

# Plot the data
plt.figure(figsize=(10, 6))
plt.bar(worm_ids, original_lengths, label='Original Lifespan', alpha=0.7)
plt.bar(worm_ids, truncated_lengths, label='Truncated Lifespan', alpha=0.7)
plt.xticks(rotation=45, ha='right')
plt.xlabel('Worms')
plt.ylabel('Number of Frames')
plt.title('Original vs Truncated Lifespan for Each Worm')
plt.legend()
plt.tight_layout()
plt.show()

# Step 3 : Standardization
> Standardization ensures that all features are on the same scale, typically with a mean of 0 and a standard deviation of 1. This helps models converge faster and makes them less sensitive to the scale of input features.
> We should avoid standar

In [ ]:
importlib.reload(helpers)

In [ ]:
# Specify the indices of the features to standardize (e.g., Speed, X, Y)
feature_columns = [1, 2, 3, 4]  # Assuming 1 = Speed, 2 = X, 3 = Y, 4 = Changed Pixels

# Apply standardization
standardized_worms = helpers.standardize_data(processed_worms, feature_columns)

In [ ]:
# Retrieve the worm data
worm_5_data = processed_worms['worm_5']  # Assuming worms is your dictionary of worm data

# Convert to a DataFrame for easier inspection
columns = ['Frame', 'Speed', 'X', 'Y', 'Changed Pixels', 'Category']
df_worm_5 = pd.DataFrame(worm_5_data.T, columns=columns)  # Transpose for proper orientation

# Display the first few rows
print(df_worm_5.head())

# Check for NaNs
print("\nChecking for NaNs...")
print(df_worm_5.isna().sum())